In [1]:
from dotenv import load_dotenv
load_dotenv('.env', override=True)

# https://python.langchain.com/docs/integrations/toolkits/openapi
from langchain_community.chat_models.azure_openai import AzureChatOpenAI
from langchain.agents.agent_toolkits.openapi import planner
from langchain.agents.agent_toolkits.openapi.spec import reduce_openapi_spec
from langchain.requests import RequestsWrapper
import requests

azure_llm = AzureChatOpenAI(azure_deployment="gpt-4")
requests_wrapper = RequestsWrapper()

res = requests.get("http://localhost:8000/openapi.json")
price_api_spec = reduce_openapi_spec(res.json())
print(price_api_spec)

cpus = []
with open('cpu.txt', encoding='utf-8') as f:
    cpus = f.read().splitlines()

ReducedOpenAPISpec(servers=[{'url': 'http://localhost:8000'}], description='This is a price query service for a private hardware vendor, which returns a quotation based on the product model', endpoints=[('POST /cpu/price', 'Given a CPU model, return the price', {'description': 'Given a CPU model, return the price', 'responses': {'description': 'Successful Response', 'content': {'application/json': {'schema': {}}}}, 'requestBody': {'content': {'application/json': {'schema': {'properties': {'model': {'type': 'string', 'title': 'Model'}}, 'type': 'object', 'required': ['model'], 'title': 'Cpu'}}}, 'required': True}}), ('POST /gpu/price', 'Given a GPU model, return the price', {'description': 'Given a GPU model, return the price', 'responses': {'description': 'Successful Response', 'content': {'application/json': {'schema': {}}}}, 'requestBody': {'content': {'application/json': {'schema': {'properties': {'model': {'type': 'string', 'title': 'Model'}}, 'type': 'object', 'required': ['model'

In [2]:
price_agent = planner.create_openapi_agent(price_api_spec, requests_wrapper, azure_llm)
query = (f"""判斷以下這是甚麼類型的產品，然後用這個產品型號去相對應的API中搜尋價格。
我想知道我的供應商是否有提供比以下這產品更便宜的價格:
{cpus[-1]}""")

In [19]:
# https://smith.langchain.com/public/9707f4e4-ae2d-4aec-b363-4d00439ec641/r
price_agent.run(query)



> Entering new AgentExecutor chain...
Action: api_planner
Action Input: I need to identify the product type for "Intel i9-14900K" and then find the corresponding API to search for its price to compare with the supplier's offering.
Observation: Plan:
1. Based on the product name "Intel i9-14900K", we can evaluate that it is a CPU model.
2. POST /cpu/price with the CPU model "Intel i9-14900K" as a parameter to get the price of the CPU for comparison with the supplier's offering.
Thought:The plan has been created, and now I need to execute the API calls.
Action: api_controller
Action Input: 1) POST /cpu/price with the CPU model "Intel i9-14900K" as a parameter to get the price of the CPU for comparison with the supplier's offering.

> Entering new AgentExecutor chain...
Thought: I need to make a POST request to the "/cpu/price" endpoint with the CPU model as a parameter to get the price.

Action: requests_post

Action Input:
```json
{
  "url": "http://localhost:8000/cpu/price",
  "data"

"The price for the CPU model Intel i9-14900K from the API is $20,000, which is cheaper than the supplier's price of $20,700."

In [2]:
from langchain.llms import Ollama

ollama_api = "https://ollama.phate.dev"
openchat_llm = Ollama(base_url=ollama_api, model="openchat-3.5-1210")
price_agent = planner.create_openapi_agent(price_api_spec, requests_wrapper, openchat_llm, agent_executor_kwargs={"handle_parsing_errors": True})
query = (f"""判斷以下這是甚麼類型的產品，然後用這個產品型號去相對應的API中搜尋價格。
我想知道我的供應商是否有提供比以下這產品更便宜的價格:
{cpus[-1]}""")

In [3]:
price_agent.run(query)



> Entering new AgentExecutor chain...
 Action: api_planner
Action Input: I need to find the right API calls to determine the product type, search for prices, and compare prices.
Observation:  1. POST /cpu/price with the CPU model as a parameter to find the price of a specific CPU.
2. POST /gpu/price with the GPU model as a parameter to find the price of a specific GPU.
3. Compare the prices obtained from steps 1 and 2 to determine which product type has a lower price or better value.
Thought: I'm ready to execute the API calls.
Action: api_controller
Action Input: 1) POST /cpu/price with the CPU model as a parameter to find the price of a specific CPU.
2) POST /gpu/price with the GPU model as a parameter to find the price of a specific GPU.
3) Compare the prices obtained from steps 1 and 2 to determine which product type has a lower price or better value.

> Entering new AgentExecutor chain...
 I need to make two POST requests, one for each endpoint, to obtain the prices of the CPUs 